## AlphaVantage Data Documentation

The following documentation goes through the finance data received from alphaVantage, its output, transformations, normalizing it, and where to stage the data in AWS s3.

Data:

- Income Statement
- Balance Sheet
- Cash Flow
- ETF Profile & Holdings
- Corporate Action


# Income Statement

In [11]:
import polars as pl
from alphaio import AlphaIO

In [2]:
# load in data for RGLD
alphaio = AlphaIO()
rgld_income = alphaio.get_statement(ticker='RGLD', statement='income')

rgld_income[0]

{'symbol': 'RGLD',
 'annualReports': [{'fiscalDateEnding': '2023-12-31',
   'reportedCurrency': 'USD',
   'grossProfit': '343394000',
   'totalRevenue': '605717000',
   'costOfRevenue': '130284000',
   'costofGoodsAndServicesSold': '90523000',
   'operatingIncome': '303202000',
   'sellingGeneralAndAdministrative': '39761000',
   'researchAndDevelopment': 'None',
   'operatingExpenses': '79953000',
   'investmentIncomeNet': 'None',
   'netInterestIncome': '-20915000',
   'interestIncome': '9952000',
   'interestExpense': '30867000',
   'nonInterestIncome': '605717000',
   'otherNonOperatingIncome': 'None',
   'depreciation': '431000',
   'depreciationAndAmortization': '191054000',
   'incomeBeforeTax': '281448000',
   'incomeTaxExpense': '42008000',
   'interestAndDebtExpense': 'None',
   'netIncomeFromContinuingOperations': '240132000',
   'comprehensiveIncomeNetOfTax': '239440000',
   'ebit': '303202000',
   'ebitda': '468139000',
   'netIncome': '239440000'},
  {'fiscalDateEnding': 

In [3]:
# print the length of the output
print(f"Length of output: {len(rgld_income)}")
print(f"Keys of the first element: {rgld_income[0].keys()}")

Length of output: 1
Keys of the first element: dict_keys(['symbol', 'annualReports', 'quarterlyReports'])


In [4]:
print(f"Length of the annualReports: {len(rgld_income[0]['annualReports'])}")
print(f"Length of the quarterlyReports: {len(rgld_income[0]['quarterlyReports'])}")

Length of the annualReports: 13
Length of the quarterlyReports: 59


In [6]:
print(f"first element of the annualReports: {rgld_income[0]['annualReports'][1]}")
print(f"first element of the quarterlyReports: {rgld_income[0]['quarterlyReports'][1]}")

first element of the annualReports: {'fiscalDateEnding': '2022-12-31', 'reportedCurrency': 'USD', 'grossProfit': '323101000', 'totalRevenue': '603206000', 'costOfRevenue': '133541000', 'costofGoodsAndServicesSold': '94642000', 'operatingIncome': '283709000', 'sellingGeneralAndAdministrative': '34612000', 'researchAndDevelopment': 'None', 'operatingExpenses': '74004000', 'investmentIncomeNet': 'None', 'netInterestIncome': '-9338000', 'interestIncome': '7832000', 'interestExpense': '17170000', 'nonInterestIncome': '603206000', 'otherNonOperatingIncome': 'None', 'depreciation': '493000', 'depreciationAndAmortization': '183222000', 'incomeBeforeTax': '271908000', 'incomeTaxExpense': '32926000', 'interestAndDebtExpense': 'None', 'netIncomeFromContinuingOperations': '239942000', 'comprehensiveIncomeNetOfTax': '238982000', 'ebit': '283709000', 'ebitda': '474763000', 'netIncome': '238982000'}
first element of the quarterlyReports: {'fiscalDateEnding': '2024-06-30', 'reportedCurrency': 'USD', '

In [8]:
df_income_annual = pl.DataFrame(rgld_income[0]['annualReports'])
df_income_quarterly = pl.DataFrame(rgld_income[0]['quarterlyReports'])

df_income_annual.head()

fiscalDateEnding,reportedCurrency,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,investmentIncomeNet,netInterestIncome,interestIncome,interestExpense,nonInterestIncome,otherNonOperatingIncome,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""2023-12-31""","""USD""","""343394000""","""605717000""","""130284000""","""90523000""","""303202000""","""39761000""","""None""","""79953000""","""None""","""-20915000""","""9952000""","""30867000""","""605717000""","""None""","""431000""","""191054000""","""281448000""","""42008000""","""None""","""240132000""","""239440000""","""303202000""","""468139000""","""239440000"""
"""2022-12-31""","""USD""","""323101000""","""603206000""","""133541000""","""94642000""","""283709000""","""34612000""","""None""","""74004000""","""None""","""-9338000""","""7832000""","""17170000""","""603206000""","""None""","""493000""","""183222000""","""271908000""","""32926000""","""None""","""239942000""","""238982000""","""283709000""","""474763000""","""238982000"""
"""2021-06-30""","""USD""","""333002000""","""648614000""","""121285000""","""92898000""","""337602000""","""28387000""","""None""","""57693000""","""None""","""-1148000""","""None""","""1148000""","""649762000""","""None""","""356000""","""None""","""339399000""","""36867000""","""1148000""","""302776000""","""302532000""","""337602000""","""None""","""302532000"""
"""2020-06-30""","""USD""","""236058000""","""498819000""","""115426000""","""83890000""","""198945000""","""30195000""","""None""","""36726000""","""None""","""-1136000""","""None""","""1136000""","""498819000""","""None""","""387000""","""None""","""195689000""","""-3654000""","""1136000""","""196250000""","""199343000""","""198945000""","""None""","""199343000"""
"""2019-06-30""","""USD""","""178553000""","""423056000""","""108023000""","""77535000""","""140707000""","""30488000""","""None""","""37646000""","""None""","""-15288000""","""None""","""15288000""","""423056000""","""None""","""200000""","""None""","""111323000""","""17498000""","""15288000""","""89079000""","""93825000""","""140707000""","""None""","""93825000"""


In [9]:
df_income_quarterly.head()

fiscalDateEnding,reportedCurrency,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,investmentIncomeNet,netInterestIncome,interestIncome,interestExpense,nonInterestIncome,otherNonOperatingIncome,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""2024-09-30""","""USD""","""129034000""","""193837000""","""37380000""","""27192000""","""118846000""","""10102000""","""None""","""10188000""","""None""","""-581000""","""626000""","""1207000""","""193837000""","""None""","""None""","""86000""","""117752000""","""21510000""","""None""","""96330000""","""96242000""","""118846000""","""118932000""","""96242000"""
"""2024-06-30""","""USD""","""112683000""","""174096000""","""34774000""","""24174000""","""102083000""","""10511000""","""None""","""10600000""","""None""","""-1709000""","""807000""","""2516000""","""174096000""","""None""","""None""","""89000""","""100199000""","""18991000""","""None""","""81320000""","""81208000""","""102083000""","""102172000""","""81208000"""
"""2024-03-31""","""USD""","""87019000""","""148902000""","""33245000""","""21751000""","""75525000""","""11412000""","""None""","""11494000""","""None""","""-1630000""","""2977000""","""4607000""","""148902000""","""None""","""None""","""82000""","""74199000""","""27033000""","""None""","""47309000""","""47166000""","""75525000""","""75607000""","""47166000"""
"""2023-12-31""","""USD""","""89516000""","""152666000""","""30180000""","""20785000""","""79690000""","""9741000""","""None""","""49587000""","""None""","""-3396000""","""2604000""","""6000000""","""152666000""","""None""","""None""","""38413000""","""76135000""","""13356000""","""None""","""62962000""","""62779000""","""79690000""","""119780000""","""62779000"""
"""2023-09-30""","""USD""","""75606000""","""138617000""","""31395000""","""21351000""","""65562000""","""9927000""","""None""","""10044000""","""None""","""-4849000""","""2436000""","""7285000""","""138617000""","""None""","""None""","""117000""","""60089000""","""10752000""","""None""","""49499000""","""49337000""","""65562000""","""65679000""","""49337000"""


## Income Statement Data Definition

https://documentation.alphavantage.co/FundamentalDataDocs/gaap_documentation.html#IncomeStatement

In [20]:
income_schema = [
    # 'fiscalDateEnding',
    # 'reportedCurrency',
    'grossProfit',
    'totalRevenue',
    'costOfRevenue',
    'costofGoodsAndServicesSold',
    'operatingIncome',
    'sellingGeneralAndAdministrative',
    'researchAndDevelopment',
    'operatingExpenses',
    'investmentIncomeNet',
    'netInterestIncome',
    'interestIncome',
    'interestExpense',
    'nonInterestIncome',
    'otherNonOperatingIncome',
    'depreciation',
    'depreciationAndAmortization',
    'incomeBeforeTax',
    'incomeTaxExpense',
    'interestAndDebtExpense',
    'netIncomeFromContinuingOperations',
    'comprehensiveIncomeNetOfTax',
    'ebit',
    'ebitda',
    'netIncome'
]

for column in income_schema:
    df_income_annual = df_income_annual.with_columns(pl.col(column).cast(pl.Float64, strict=False))
    df_income_quarterly = df_income_quarterly.with_columns(pl.col(column).cast(pl.Float64, strict=False))

df_income_annual = df_income_annual.with_columns(pl.col('fiscalDateEnding').str.to_date('%Y-%m-%d'))
df_income_quarterly = df_income_quarterly.with_columns(pl.col('fiscalDateEnding').str.to_date('%Y-%m-%d'))
df_income_annual.head()

fiscalDateEnding,reportedCurrency,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,investmentIncomeNet,netInterestIncome,interestIncome,interestExpense,nonInterestIncome,otherNonOperatingIncome,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome
date,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2023-12-31,"""USD""",3.43394e8,6.05717e8,1.30284e8,9.0523e7,3.03202e8,3.9761e7,null,7.9953e7,null,-2.0915e7,9.952e6,3.0867e7,6.05717e8,null,431000.0,1.91054e8,2.81448e8,4.2008e7,null,2.40132e8,2.3944e8,3.03202e8,4.68139e8,2.3944e8
2022-12-31,"""USD""",3.23101e8,6.03206e8,1.33541e8,9.4642e7,2.83709e8,3.4612e7,null,7.4004e7,null,-9.338e6,7.832e6,1.717e7,6.03206e8,null,493000.0,1.83222e8,2.71908e8,3.2926e7,null,2.39942e8,2.38982e8,2.83709e8,4.74763e8,2.38982e8
2021-06-30,"""USD""",3.33002e8,6.48614e8,1.21285e8,9.2898e7,3.37602e8,2.8387e7,null,5.7693e7,null,-1.148e6,null,1.148e6,6.49762e8,null,356000.0,null,3.39399e8,3.6867e7,1.148e6,3.02776e8,3.02532e8,3.37602e8,null,3.02532e8
2020-06-30,"""USD""",2.36058e8,4.98819e8,1.15426e8,8.389e7,1.98945e8,3.0195e7,null,3.6726e7,null,-1.136e6,null,1.136e6,4.98819e8,null,387000.0,null,1.95689e8,-3.654e6,1.136e6,1.9625e8,1.99343e8,1.98945e8,null,1.99343e8
2019-06-30,"""USD""",1.78553e8,4.23056e8,1.08023e8,7.7535e7,1.40707e8,3.0488e7,null,3.7646e7,null,-1.5288e7,null,1.5288e7,4.23056e8,null,200000.0,null,1.11323e8,1.7498e7,1.5288e7,8.9079e7,9.3825e7,1.40707e8,null,9.3825e7
